In [31]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Generating Card Games

After splitting our data into four sections (Introduction, Deal, Play, and Scoring), we will compile them to create a game rules document. We will be applying a recurrent neural network and LTSM to achieve this. 

## Preprocessing
First, we will import and clean our data.

In [18]:
data = pd.read_csv('../data/text_data_grouped_by_cat.csv').drop('Unnamed: 0', axis=1).drop(8091).drop_duplicates()

# Clean data
data['Text'] = (data['Text']
                   .apply(lambda x: x.lower()
                                      .replace('\n', '')))

In [19]:
# Create subsets
introductions = data.loc[data['index'] == 'Introduction']
deal = data.loc[data['index'] == 'Deal']
play = data.loc[data['index'] == 'Play']
scoring = data.loc[data['index'] == 'Scoring']

## Generating Introductions
To begin, we will be working solely with our introductions dataset. We are referencing [this notebook](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_10_3_text_generation.ipynb), which references the keras documentation.

In [20]:
# define parameters
maxlen = 100
step = 3
BATCH_SIZE = 64
epochs = 60

## Preprocessing
First, we create a function to process the text. For the sake of this project, we are only going to keep ascii characters.

In [21]:
def process_text(df):   
    string = ''
    for i in df["Text"]:
        string+=i
    # keep only ascii
    return re.sub(r'[^\x00-\x7f]',r'', string)

In [22]:
processed_text = process_text(introductions)

## Setup
Next, we are going to create a set up function. The function performs the following actions:
1. create a dictionary to map characters to numbers
2. divide our text into sample sequences to train our model on
3. vectorize our sequences into matrix form

In [23]:
def setup(processed_text, maxlen, step):
    
    # create dictionary
    chars = sorted(list(set(processed_text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    
    # divides text into sample sequences
    sentences = []
    next_chars = []
    for i in range(0, len(processed_text) - maxlen, step):
        sentences.append(processed_text[i: i + maxlen])
        next_chars.append(processed_text[i + maxlen])
    
    # vectorize into matrix
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test, sentences, char_indices, indices_char, chars

In [24]:
# Split data into train/test
X_train, X_test, y_train, y_test, sentences, char_indices, indices_char, chars = setup(processed_text, maxlen, step)

## Build Model
We now are ready to build our model. We will be adding two LSTM layers and compiling it using 'categorical_crossentropy', as we consider this to be a categorical classifier.

In [25]:
model = Sequential()
model.add(LSTM(BATCH_SIZE, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(BATCH_SIZE))
model.add(Dense(len(chars), activation='softmax'))
#model.add(tf.keras.layers.Dropout(0.4))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100, 64)           32256     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 61)                3965      
Total params: 69,245
Trainable params: 69,245
Non-trainable params: 0
_________________________________________________________________


## Train
Lastly, we train our model. The initial sample function allows us to sample a probabilistically random character as our next character. Next, we will display the trained model with temperatures [0.2, 0.5, 1.0, 1.2]. We've made the minimum epoch 20, but otherwise will continue to run to 60 epochs as the validation error decreases

In [27]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [32]:
def on_epoch_end(epoch, _):
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
    print('\nhistory dict:', model.history)


In [33]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=epochs,
          callbacks=[print_callback, es_callback],
          validation_data=(X_test, y_test))

Epoch 1/60
946/946 [==============================] - ETA: 0s - loss: 2.2147****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "  and the aim of the game is to accumulate a score of 1000 points or more. 13-card brag this simulat"
  and the aim of the game is to accumulate a score of 1000 points or more. 13-card brag this simulat and and a cands and the tack in the game players and and and and with trick conte players to cand and and the game to the segent the game and and and is a trick in the game the and game players of the game the game the to the near the sein the game is the tack to cards to the seat the game bet in the game of the game to the to the secare the game the game with trick to be the game of the game the
----- temperature: 0.5
----- Generating with seed: "  and the aim of the game is to accumulate a score of 1000 points or more. 13-card brag this simulat"
 

rmined - say for example $5 and $10. when there is a compulsory bring-in bet, the ante amount is genaza is mealabhap3 ond so mow or cuntbirnou. boono game onl weaks. verenn or condad thate by mibuly which it secland valwe onloarce with "onlra and premanflet a grouok, sluwsg.it it sention uil sctebopz. popular betting barssucia consen - dilced tuncly kentu havame eursimiined in a 0oventutiand and in royf-yorolco, this poguosy pages on seotettaged on a partrades, plaino or popi forlo were ligingn"

history dict: <tensorflow.python.keras.callbacks.History object at 0x148756350>
946/946 [==============================] - 137s 145ms/step - loss: 1.6535 - val_loss: 1.7254
Epoch 4/60
946/946 [==============================] - ETA: 0s - loss: 1.5676****************************************************************************
----- Generating text after Epoch: 3
----- temperature: 0.2
----- Generating with seed: "from  cleveland, ohio. i do not know how old it is, how widely it is played, whethe

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


ge of the last played in the different game is a card game in the players in the played in the name is a several page is a simi
----- temperature: 0.5
----- Generating with seed: "at card game for some years a faq by richard sipie, first published in 2000, was the only generally "
at card game for some years a faq by richard sipie, first published in 2000, was the only generally played in played in the relation of this page of this page betting game in the was southern cards. poker point to the game is a very in the players cards played in no compule in  simple the name in which the game is a point of this game is the table and which the hand on this game in canastia player version of this game boondine the card game of this page in this game boing for the poker the nines
----- temperature: 1.0
----- Generating with seed: "at card game for some years a faq by richard sipie, first published in 2000, was the only generally "
at card game for some years a faq by richard sipie, first publi

Epoch 9/60
946/946 [==============================] - ETA: 0s - loss: 1.3806****************************************************************************
----- Generating text after Epoch: 8
----- temperature: 0.2
----- Generating with seed: "amata this popular five-player version of the italian game briscola differs from the parent game in "
amata this popular five-player version of the italian game briscola differs from the parent game in the originated in the game is a game is that the game is a players in the game is also played in the game is a players and in the game is that the game is a partnerships and in the game is a hand of the game is a partnerships and in the game is a probably played with the game is a probably the game is a players to the game is a partnerships and in the game is a can be a partnership that the game i
----- temperature: 0.5
----- Generating with seed: "amata this popular five-player version of the italian game briscola differs from the parent game in "
a

...

hip game in which each player has just one hand. there are no partnerships in new canasta, the "handbskope hreplasing cothert of the jass sometim in.  partnership varion of this pugher this tablet-ceart  provide attlivision skrytcher. at the turnhent of three-card dra poker for french, this is a edd stasin hial 'sioble back white end dovhs for north america, . card popular the players jist for nathhied in page bankert mean. in chinese time.  disky coolettife, which in each onle pole-the refult t
----- temperature: 1.2
----- Generating with seed: "hip game in which each player has just one hand. there are no partnerships in new canasta, the "hand"
hip game in which each player has just one hand. there are no partnerships in new canasta, the "handuo if egplate of popularals whirestlumes belesenker dos blarire de (19eais prossonco.ma1 ens thatbe hermie ()s, bias game derfa croatchewing cards mone or ntirbyten by kistay hand uld a trins apparent from hneveral your dange that tulock", pimil

s properly known as huutopussi ("bidding bag") and is now described on a separate huutopussi page. the name is fourlerning and the durag and all player maked is played in the name rummy or in the repeship to the lest other player is dealting games played in the hinds in partner and the first parts of the the learng lay the fartinase of the know this page is a sumbes are allowing page. the obsic game is a spone and this game is a single the game is also played in the english game has austria has 
----- temperature: 1.0
----- Generating with seed: "s properly known as huutopussi ("bidding bag") and is now described on a separate huutopussi page. t"
s properly known as huutopussi ("bidding bag") and is now described on a separate huutopussi page. the cards in the ferds of a locu may people cards bet severan is version. sxed by a king to the spoble the casino kryt0y beach doirdo contributed are used the direman luparica agains a seghtion domine for a each hand is also upino this games butk

game of ireland: it is played widely in rural pubs and at home. the game moves quite quickly - it don the game of the game of the game of the game of the game of the game of the game of the game of the game of the game of the game of the game of the game of the bull cards in the name of the game of the game of the game of the game of the game is a nulest that the game of the game of the game of the game of the game of the game is a sourgh the game of the game of the game of the the game of the g
----- temperature: 0.5
----- Generating with seed: "game of ireland: it is played widely in rural pubs and at home. the game moves quite quickly - it do"
game of ireland: it is played widely in rural pubs and at home. the game moves quite quickly - it don, and the route of the bumit the alter other described by the last value in amering game go doise thither are lowen the game of the turplinged in the last the both centrally span2sk - reparition to which is also played by alternative  i am utho

t ibragim khusim from the town of sana, yemen, who had played basra for several years, also with a tracteg of  yoo, the lankeritiis is leftented it, speaking the name information frou1 straiu- hand", riscan fet 90 limeteu clarme1 oubss, use ma7nubka" varias and asinouu can in omiout subno vougres nower harst pater as vould recenamen for jac openst partner big twod, is a rummy sourd to cikes you miinut has a variants, hict are  deruent-jaken, the obbestend the tworces, ard and played in priida, s

history dict: <tensorflow.python.keras.callbacks.History object at 0x147687c90>
946/946 [==============================] - 129s 137ms/step - loss: 1.2385 - val_loss: 1.7793
Epoch 23/60
946/946 [==============================] - ETA: 0s - loss: 1.2274****************************************************************************
----- Generating text after Epoch: 22
----- temperature: 0.2
----- Generating with seed: "prises.com the main game described on this page is three player single deck aucti

italian card game burracothe argentinean tile game burako the italian card game burraco burraco the variantred youg high-gamesesskut or shain a'ccunge, bmavu for benti3 or  variant pater name i have one is the use, this neerss in your card games, with have the purears where is startiction of termingland , the netherlation is a vrouct is a game not and poker trost cards cards, but there are allowed, and proui by the draw complarial reaned pedertinationally rummy aclound movetted are deterested by
----- temperature: 1.2
----- Generating with seed: "italian card game burracothe argentinean tile game burako the italian card game burraco burraco the "
italian card game burracothe argentinean tile game burako the italian card game burraco burraco the largishpraws begins where another version gildar invoncerean some them hiding though formry its jarsina arrather original suits and paii oppes to the playerss cards areer and to old points, forcup that - ,qafopal ayi2. juhs. ivee pobre", have ea

## Predict

In [39]:
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('----- temperature:', temperature)

    generated = ''
    sentence = "this rapid two-player game requires accurate"
    generated += sentence
    sys.stdout.write(generated)

    for i in range(600):
        x_pred = np.zeros((1, len(sentence), len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

----- temperature: 0.2
this rapid two-player game requires accurate this page a

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 popular and it is also played in several partner and and it is a points of the game is played in the name of the game is played in a similar to the games of the game is played by all the descriptions of the game is also played in the name of the games and the game is played in the name of the game is also also also some a more than the games the name of the game is also played in the name of the games and the games of the game is also played in the name of the game is also played in several poker cards are played by the name of the game is also played in each players and the objec
----- temperature: 0.5
this rapid two-player game requires accurate canasta and the game is played to the game is a litter , substable scoring the oent that the following southed by the descriptions of pairisions of the name will amiet to west game of cards bookle as in the all dect of the games. the game of cards. notes and foot is interesting page, and the game of the name of the description and points, al